In [87]:
using GMT
using LibGEOS
using Plots

include("RhumbLinesCalculations.jl")
include("HandleTrackFunctions.jl")

x_data=[1,5.1,36.1,11,6.2];
y_data=[36, 37, 38, 39,38];
z_data=[-1,-2,-3,-4,0];
Mw_data=[1,2,3,4,1];
start_end_points=[5,36,10,48]
Npoints_GMT=500
w=100

distances,total_distance,lon_rect,lat_rect,azimuth_,dataOnTrack=compute_tracks(w,start_end_points,Npoints_GMT)

lon_lat_rect=[lon_rect lat_rect]

# compute min and max values of the rectangle in order to fasten the computaiton

lon_min=minimum(lon_rect);
lon_max=maximum(lon_rect);
lat_min=minimum(lat_rect);
lat_max=maximum(lat_rect);

cond=((x_data .>= lon_min) .& (x_data .<= lon_max)) .& ((y_data .>= lat_min) .& (y_data .<= lat_max));

x_data=x_data[cond];
y_data=y_data[cond];
z_data=z_data[cond];
Mw_data=Mw_data[cond];

###

polycontour=[lon_lat_rect[k,:] for k in 1:size(lon_lat_rect,1)];
SwathPolygon=LibGEOS.Polygon([polycontour]);

Plots.plot(SwathPolygon);

withinPoly=Bool[]
for i=1:length(x_data)
    onePoint=LibGEOS.Point(x_data[i],y_data[i]);
    push!(withinPoly,LibGEOS.intersects(onePoint,SwathPolygon));
end

x_data=x_data[withinPoly];
y_data=y_data[withinPoly];
z_data=z_data[withinPoly];
Mw_data=Mw_data[withinPoly];

int_points_lon=Float64[];
int_points_lat=Float64[];
disances_points=Float64[];

for i=1:length(x_data)
    int_point_lat,int_point_lon=rhxrh(y_data[i], x_data[i], azimuth_, start_end_points[2], start_end_points[1], azimuth_-90)
    push!(int_points_lon,int_point_lon)
    push!(int_points_lat,int_point_lat)
    push!(disances_points,rhumb_distance([x_data[i], y_data[i]], [int_point_lon, int_point_lat]))
end

GMT.plot(region=[4,21,35,49],x=dataOnTrack[:,1],y=dataOnTrack[:,2],J="M15c")
GMT.scatter!(x=x_data,y=y_data)
GMT.scatter!(x=int_points_lon,y=int_points_lat)
GMT.plot!(x=lon_rect,y=lat_rect,J="M15c",pen="0.1p,red")
GMT.coast!(name="temp.pdf")